# Snowflake Horizon Catalog & Internal Marketplace Hands-on
本ハンズオンでは、以下の2つの構成に分かれてハンズオンを実施します。それぞれのパートで機能を実感していだき概要を理解していただくことを目的としています。
1. Snowflake Horizon Catalog
2. Snowflake Internal Marketplace

## Snowflake Horizon Catalog

### Universal Search
SnowflakeのHorizon Catalogでは、格納されている資産(テーブル、ビュー、ワークシート、ダッシュボード、ノートブック、Streamlitアプリ、MLモデルなど)を横断して検索・発見するための機能を提供しています。
それでは、実際に検索を行ってみます。

#### Step 1 - キーワード検索
1. Snowsightの検索/Searchアイコンをクリック
2. 検索窓に`truck`と入力します。
3. テーブルおよびビュー/Tables and viewsの検索結果として、truckの項目を含む一覧が表示されます

### Step 2 - 文章検索
キーワードだけではなく、文章から検索を行うことも可能です。
1. 次のように検索ワードを入力してみてください `Which truck franchises has the most loyal customer base?`
2. 検索条件を満たすために必要となりそうなテーブル/Viewを検索結果として表示してくれます

### AIを活用したオブジェクト説明
Snowflake Copilot(AI)を活用して、テーブルやビューにコメントを即座に生成可能

#### Step 1 - UIを使ったオブジェクト説明
1. Snowsightから、カタログ/Catalog > データベースエクスプローラー/Database Explorerを開く
2. TB_101 > RAW_POS > ORDER_HEADERを選択
3. 説明/Descriptionにある`Copilotで生成/Generate with Cortex`をクリック

### Step 2 - SQLを使ったオブジェクト生成
[AI_GENERATE_TABLE_DESC](https://docs.snowflake.com/en/sql-reference/stored-procedures/ai_generate_table_desc)関数を利用して、各列単位での説明も生成することが可能です。
これにより[サンプル](https://docs.snowflake.com/user-guide/sql-cortex-descriptions)のように自動的に説明が追加されるように設定することも可能です。

In [ ]:
from snowflake.snowpark.context import get_active_session
import json
import streamlit as st

session = get_active_session()
session.use_role("sysadmin")

TABLE_NAME = "CUSTOMER_LOYALTY"
SCHEMA_NAME = "RAW_CUSTOMER"
DATABASE_NAME = "TB_101"

# 説明生成
df_result = session.sql(f"""
  CALL AI_GENERATE_TABLE_DESC('{DATABASE_NAME}.{SCHEMA_NAME}.{TABLE_NAME}',
    {{
        'describe_columns': true,
        'use_table_data': true
    }}
  )
""").to_pandas()

# 取得結果をオブジェクトに変換
output = json.loads(df_result.iat[0,0])
columns_ret = output["COLUMNS"]
table_ret = output["TABLE"][0]

# テーブル説明を取得し、エスケープ処理
table_description = table_ret["description"]
table_name = table_ret["name"]
database_name = table_ret["database_name"]
schema_name = table_ret["schema_name"]

# テーブル説明を更新
table_description = table_description.replace("'", "\\'")
session.sql(f"""ALTER TABLE {database_name}.{schema_name}.{table_name} SET COMMENT = '{table_description}'""").collect()

# 各カラムの値を取得し、更新
for column in columns_ret:
    column_description = column["description"];
    column_name = column["name"];
    if not column_name.isupper():
        column_name = '"' + column_name + '"'
    
    column_description = column_description.replace("'", "\\'")
    session.sql(f"""ALTER TABLE  {database_name}.{schema_name}.{table_name} MODIFY COLUMN {column_name}  COMMENT '{column_description}'""").collect()

st.markdown("""
処理が完了したので、結果を確認しみましょう。
1. Snowsightから、カタログ/Catalog > データベースエクスプローラー/Database Explorerを開く
2. TB_101 > RAW_CUSTOMER > テーブル > CUSTOMER_LOYALTYを選択
3. 説明/Descriptionに説明が入っていることを確認
4. 列/Columnsを選択し、各カラムに説明が入っていることを確認
""")

### 分類/タグ付
データガバナンスにおける重要な第一歩は、機密データの特定と分類です。Snowflake Horizonの自動タグ付け機能は、スキーマ内のカラムを監視することで機密情報を自動的に検出します。これらのタグを活用してセキュリティポリシーを適用することが可能です。

#### Step 1 - PIIタグを作成し権限を付与
accountadminロールを使用して、ガバナンススキーマ内にpiiタグを作成します。また、分類処理を実行するためにtb_data_stewardロールに必要な権限を付与します。

In [ ]:
USE ROLE accountadmin;
USE DATABASE TB_101;

CREATE OR REPLACE TAG governance.pii;
GRANT APPLY TAG ON ACCOUNT TO ROLE tb_data_steward;

GRANT EXECUTE AUTO CLASSIFICATION ON SCHEMA raw_customer TO ROLE tb_data_steward;
GRANT DATABASE ROLE SNOWFLAKE.CLASSIFICATION_ADMIN TO ROLE tb_data_steward;
GRANT CREATE SNOWFLAKE.DATA_PRIVACY.CLASSIFICATION_PROFILE ON SCHEMA governance TO ROLE tb_data_steward;

#### Step 2 - 分類プロファイルの作成
`tb_data_steward`ロールを利用して、分類プロファイルを作成します。このプロファイルは自動タギングの振る舞いを定義しています。

In [ ]:
USE ROLE tb_data_steward;

CREATE OR REPLACE SNOWFLAKE.DATA_PRIVACY.CLASSIFICATION_PROFILE
  governance.tb_classification_profile(
    {
      'minimum_object_age_for_classification_days': 0,
      'maximum_classification_validity_days': 30,
      'auto_tag': true
    });

#### Step 3 - セマンティックカテゴリをPIIタグにマッピング
次に、分類プロファイルに対して、SEMANTIC_CATEGORY が NAME、PHONE_NUMBER、EMAIL などの一般的な個人識別情報（PII）タイプと一致する列に、当社の governance.pii タグを適用するよう指示するマッピングを定義します。

In [ ]:
CALL governance.tb_classification_profile!SET_TAG_MAP(
  {'column_tag_map':[
    {
      'tag_name':'tb_101.governance.pii',
      'tag_value':'pii',
      'semantic_categories':['NAME', 'PHONE_NUMBER', 'POSTAL_CODE', 'DATE_OF_BIRTH', 'CITY', 'EMAIL']
    }]});

#### Step 4 - 分類の実行と結果確認
顧客ロイヤルティテーブルに対して分類プロセスを手動でトリガーしましょう。その後、INFORMATION_SCHEMAをクエリして自動的に適用されたタグを確認できます。

In [ ]:
-- Trigger classification
CALL SYSTEM$CLASSIFY('tb_101.raw_customer.customer_loyalty', 'tb_101.governance.tb_classification_profile');

-- View applied tags
SELECT 
    column_name,
    tag_database,
    tag_schema,
    tag_name,
    tag_value,
    apply_method
FROM TABLE(INFORMATION_SCHEMA.TAG_REFERENCES_ALL_COLUMNS('raw_customer.customer_loyalty', 'table'));

PIIとして識別された列には、上記のカスタムガバナンスタグ「governance.pii」が適用されています

### データ品質モニタリング
データガバナンスはセキュリティだけでなく、信頼性と確実性も重要です。Snowflakeはデータメトリック関数（DMF）によりデータの完全性を維持します。システム定義のDMFを利用するか、独自のDMFを作成してテーブルの自動品質チェックを実行できます。

#### 期待値を設定して、定期実行
Snowflakeの組み込みDMFをいくつか使用して、order_headerテーブルの品質を確認しましょう。

In [ ]:
GRANT EXECUTE DATA METRIC FUNCTION ON ACCOUNT TO ROLE sysadmin;
USE ROLE sysadmin;

-- モニタリング頻度
ALTER TABLE TB_101.RAW_POS.ORDER_HEADER SET DATA_METRIC_SCHEDULE = '5 minutes';

ALTER TABLE TB_101.RAW_POS.ORDER_HEADER
  ADD DATA METRIC FUNCTION
    -- Row count (Volume)
    SNOWFLAKE.CORE.ROW_COUNT ON () Expectation Volume_Check (value > 1),       
    -- Freshness
    SNOWFLAKE.CORE.FRESHNESS ON () Expectation Freshness_Check (value < 1800), 
    -- Null count
    SNOWFLAKE.CORE.NULL_COUNT ON (ORDER_ID) Expectation Null_check (value = 0),    
    -- DUPLICATION
    SNOWFLAKE.CORE.DUPLICATE_COUNT ON (ORDER_ID) Expectation duplicate_check (value < 1); 

In [ ]:
SELECT *
  FROM TABLE(SYSTEM$EVALUATE_DATA_QUALITY_EXPECTATIONS(
      REF_ENTITY_NAME => 'tb_101.raw_pos.order_header'))
;


### 合成データ生成
Snowflakeはソーステーブルから合成データを生成でき、ソーステーブルと同じ列数を持つテーブルを作成しますが、統計的に類似した人工データで構成されます。合成データを使用することで、機密性が高く共有が制限されているデータや、共有が不適切なデータを共有・テストできます。合成データセットは、列の名前、数、データ型など、ソースデータセットと同じ特性を持ち、行数は同じかそれ以下です。特に元のデータが機密性が高く、権限のないユーザーがアクセスすべきでない場合に、Snowflake内のワークロードをテストおよび検証するために合成データを使用できます。合成データは[データリネージグラフ](https://docs.snowflake.com/ja/user-guide/ui-snowsight-lineage)に表示されます。

In [ ]:
-- データ量が多いためサンプリングテーブルを作成
use database tb_101;
CREATE OR REPLACE SCHEMA sampled_sc;

CREATE OR REPLACE TABLE sampled_sc.order_header as select * from tb_101.raw_pos.order_header sample (5000 rows);
CREATE OR REPLACE TABLE sampled_sc.order_detail as select * from tb_101.raw_pos.order_detail sample (5000 rows);

In [ ]:
-- 合成データ格納先のスキーマを作成
CREATE SCHEMA syn_schema;

-- 複数テーブルでの結合に利用するキーを保管するためのシークレットを作成
CREATE OR REPLACE SECRET tb_101.syn_schema.consistency_secret
  TYPE=SYMMETRIC_KEY
  ALGORITHM=GENERIC;

In [ ]:
-- 合成データを作成
CALL SNOWFLAKE.DATA_PRIVACY.GENERATE_SYNTHETIC_DATA({
  'datasets':[
      {
        'input_table': 'TB_101.sampled_sc.ORDER_HEADER',
        'output_table': 'TB_101.syn_schema.ORDER_HEADER',
        'columns': { 'order_id': {'join_key': TRUE}, 'customer_id': {'join_key': TRUE}, 'location_id': {'join_key': TRUE}, 'truck_id': {'join_key': TRUE}}
      }
    ],
    'consistency_secret': SYSTEM$REFERENCE('SECRET', 'syn_schema.consistency_secret', 'SESSION', 'READ')::STRING,
    'replace_output_tables': TRUE
});

CALL SNOWFLAKE.DATA_PRIVACY.GENERATE_SYNTHETIC_DATA({
  'datasets':[      
      {
        'input_table': 'TB_101.sampled_sc.ORDER_DETAIL',
        'output_table': 'TB_101.syn_schema.ORDER_DETAIL',
        'columns': { 'order_id': {'join_key': TRUE}, 'menu_item_id': {'join_key': TRUE}}
      }
    ],
    'consistency_secret': SYSTEM$REFERENCE('SECRET', 'syn_schema.consistency_secret', 'SESSION', 'READ')::STRING,
    'replace_output_tables': TRUE
});

CALL SNOWFLAKE.DATA_PRIVACY.GENERATE_SYNTHETIC_DATA({
  'datasets':[
    {
        'input_table': 'TB_101.RAW_POS.MENU',
        'output_table': 'TB_101.syn_schema.MENU',
        'columns': { 'menu_item_id': {'join_key': TRUE}, 'menu_type_id': {'join_key': TRUE}}
      },
      {
        'input_table': 'TB_101.RAW_POS.TRUCK',
        'output_table': 'TB_101.syn_schema.TRUCK',
        'columns': { 'franchise_id': {'join_key': TRUE}, 'menu_type_id': {'join_key': TRUE}, 'truck_id': {'join_key':TRUE} }
      },
      {
        'input_table': 'TB_101.RAW_POS.LOCATION',
        'output_table': 'TB_101.syn_schema.LOCATION',
        'columns': { 'location_id': {'join_key': TRUE}}
      }      
    ],
    'consistency_secret': SYSTEM$REFERENCE('SECRET', 'syn_schema.consistency_secret', 'SESSION', 'READ')::STRING,
    'replace_output_tables': TRUE
});

CALL SNOWFLAKE.DATA_PRIVACY.GENERATE_SYNTHETIC_DATA({
  'datasets':[
      {
        'input_table': 'TB_101.RAW_POS.FRANCHISE',
        'output_table': 'TB_101.syn_schema.FRANCHISE',
        'columns': { 'franchise_id': {'join_key': TRUE}}
      },
      {
        'input_table': 'TB_101.RAW_CUSTOMER.CUSTOMER_LOYALTY',
        'output_table': 'TB_101.syn_schema.CUSTOMER_LOYALTY',
        'columns': { 
            'customer_id': {'join_key': TRUE}
        }
      }
    ],
    'consistency_secret': SYSTEM$REFERENCE('SECRET', 'syn_schema.consistency_secret', 'SESSION', 'READ')::STRING,
    'replace_output_tables': TRUE
});

#### 合成データの分布状況を確認
実際に作成された合成データと元データの分布について確認してみましょう。

In [ ]:
import streamlit as st
import altair as alt
from snowflake.snowpark.context import get_active_session
import pandas as pd

session = get_active_session()

# テーブル一覧を取得
tables = session.sql("SHOW TABLES in tb_101.syn_schema").collect()
table_names = [row['name'] for row in tables]

# テーブル選択用のドロップダウン
selected_table = st.selectbox("テーブルを選択してください", table_names)

@st.cache_data(ttl=120)
def get_synthetic_compdata(table_name):
    # テーブルのデータを取得
    df = session.table(selected_table).to_pandas()
    schema_name = "RAW_POS"
    if selected_table == "CUSTOMER_LOYALTY":
        schema_name = "RAW_CUSTOMER"
    elif selected_table in ["ORDER_HEADER", "ORDER_DETAIL"]:
        schema_name = "SYN_SCHEMA"
        
    df_org = session.table(f"TB_101.{schema_name}.{selected_table}").to_pandas()

    return df, df_org

if selected_table:
    with st.spinner("データ取得中..."):
        df, df_org = get_synthetic_compdata(selected_table)
        # # テーブルのデータを取得
        # df = session.table(selected_table).to_pandas()
        # schema_name = "RAW_POS"
        # if selected_table == "CUSTOMER_LOYALTY":
        #     schema_name = "RAW_CUSTOMER"
        # elif selected_table in ["ORDER_HEADER", "ORDER_DETAIL"]:
        #     schema_name = "SYN_SCHEMA"
            
        # df_org = session.table(f"TB_101.{schema_name}.{selected_table}").to_pandas()
        
        # 文字列型の列のみを抽出
        string_cols = df.select_dtypes(include=['object']).columns
        
        if len(string_cols) > 0:
            # 列の選択
            selected_column = st.selectbox("列を選択してください", string_cols)
    
            col_1, col_2 = st.columns(2)
            with col_1:
                st.subheader("合成データ")
                # 選択された列の値を集計
                value_counts = df[selected_column].value_counts().reset_index()
                value_counts.columns = ['VALUE', 'COUNT']
                
                # 棒グラフの作成
                chart = alt.Chart(value_counts).mark_bar().encode(
                    x=alt.X('VALUE:N', sort='-y'),
                    y='COUNT:Q',
                    tooltip=['VALUE', 'COUNT']
                ).properties(
                    title=f"{selected_table}.{selected_column}の値の分布",
                    width=600,
                    height=400
                )
                
                st.altair_chart(chart)
                
                # 集計結果の表示
                st.subheader("集計結果")
                st.dataframe(value_counts)
                
            with col_2:
                st.subheader("元データ")
                # 選択された列の値を集計
                org_value_counts = df_org[selected_column].value_counts().reset_index()
                org_value_counts.columns = ['VALUE', 'COUNT']
                
                # 棒グラフの作成
                org_chart = alt.Chart(value_counts).mark_bar().encode(
                    x=alt.X('VALUE:N', sort='-y'),
                    y='COUNT:Q',
                    tooltip=['VALUE', 'COUNT']
                ).properties(
                    title=f"{selected_table}.{selected_column}の値の分布",
                    width=600,
                    height=400
                )
                
                st.altair_chart(org_chart)
                
                # 集計結果の表示
                st.subheader("集計結果")
                st.dataframe(org_value_counts)
        else:
            st.warning("選択したテーブルに文字列型の列が存在しません")

## Internal Marketplace

Snowflake Internal Marketplaceは、企業が文書化されガバナンスが適用されたデータ製品を公開できるようにし、データ利用者がそれらを発見・理解できるようにします。オプションでデータ品質メトリクスやSLOを含めることで、製品の信頼性を高めることが可能です。また、このマーケットプレイスはデータ製品へのアクセス管理や詳細なガバナンス設定を包括的に提供する豊富な機能を備えており、どの利用者がどのデータ製品またはデータ製品の一部を利用できるかを制御できます。

### Step 1 - アカウントの設定
Internal Marketplaceで共有を行うためのロールを作成し、権限を付与します

In [ ]:
USE ROLE accountadmin;

-- Use the same name and email for all accounts
set email_var = '{{email_var}}';
set firstname_var  = '{{firstname_var}}';
set lastname_var  = '{{lastname_var}}';

-- Use the same password for users in all accounts
set pwd_var = '{{password_var}}';


-- CREATE OR REPLACE WAREHOUSE compute_wh WAREHOUSE_SIZE=xsmall INITIALLY_SUSPENDED=TRUE;
-- GRANT ALL ON WAREHOUSE compute_wh TO ROLE public;

-- --  Create a user and role for the sales domain:
USE ROLE accountadmin;
CREATE OR REPLACE ROLE sales_data_scientist_role;

SET my_user_var = CURRENT_USER();
ALTER USER identifier($my_user_var) SET DEFAULT_ROLE = sales_data_scientist_role;

GRANT CREATE SHARE ON ACCOUNT                    TO ROLE sales_data_scientist_role;
GRANT CREATE ORGANIZATION LISTING ON ACCOUNT     TO ROLE sales_data_scientist_role;


-- Next, create a user and role for the marketing domain:
USE ROLE accountadmin;
CREATE OR REPLACE ROLE marketing_analyst_role;

GRANT CREATE SHARE ON ACCOUNT                    TO ROLE marketing_analyst_role;
GRANT CREATE ORGANIZATION LISTING ON ACCOUNT     TO ROLE marketing_analyst_role;

USE ROLE orgadmin;
GRANT MANAGE LISTING AUTO FULFILLMENT ON ACCOUNT TO ROLE sales_data_scientist_role;
GRANT MANAGE LISTING AUTO FULFILLMENT ON ACCOUNT TO ROLE marketing_analyst_role;

set my_user_var = current_user();
GRANT ROLE sales_data_scientist_role to user IDENTIFIER($my_user_var);
GRANT ROLE marketing_analyst_role to user IDENTIFIER($my_user_var);

-- 上記で作成したロールに対してのアクセス権限を設定します。
USE ROLE SYSADMIN;
GRANT ALL ON DATABASE TB_101 to role sales_data_scientist_role;
GRANT ALL ON DATABASE TB_101 to role marketing_analyst_role;
GRANT ALL ON ALL SCHEMAS in DATABASE TB_101 to role sales_data_scientist_role;
GRANT ALL ON ALL SCHEMAS in DATABASE TB_101 to role marketing_analyst_role;

GRANT ALL ON ALL TABLES IN SCHEMA TB_101.RAW_CUSTOMER to role sales_data_scientist_role;
GRANT ALL ON ALL TABLES IN SCHEMA TB_101.RAW_CUSTOMER to role marketing_analyst_role;
GRANT ALL ON ALL TABLES IN SCHEMA TB_101.RAW_POS to role sales_data_scientist_role;
GRANT ALL ON ALL TABLES IN SCHEMA TB_101.RAW_POS to role marketing_analyst_role;
GRANT ALL ON ALL TABLES IN SCHEMA TB_101.RAW_SUPPORT to role sales_data_scientist_role;
GRANT ALL ON ALL TABLES IN SCHEMA TB_101.RAW_SUPPORT to role marketing_analyst_role;
GRANT ALL ON ALL TABLES IN SCHEMA TB_101.SYN_SCHEMA to role sales_data_scientist_role;
GRANT ALL ON ALL TABLES IN SCHEMA TB_101.SYN_SCHEMA to role marketing_analyst_role;
GRANT ALL ON ALL VIEWS IN SCHEMA TB_101.HARMONIZED to role sales_data_scientist_role;
GRANT ALL ON ALL VIEWS IN SCHEMA TB_101.HARMONIZED to role marketing_analyst_role;
GRANT ALL ON ALL VIEWS IN SCHEMA TB_101.ANALYTICS to role sales_data_scientist_role;
GRANT ALL ON ALL VIEWS IN SCHEMA TB_101.ANALYTICS to role marketing_analyst_role;


-- GRANT ROLE to current user
GRANT ROLE sales_data_scientist_role to user 



### Step 2 - 追加アカウントの作成
-- 共有先となるアカウントと組織アカウントを作成します。

In [ ]:
-- Run this code in your PRIMARY account
-- Create a secondary account in the same region (default!):
USE ROLE orgadmin;

CREATE ACCOUNT hol_account2
  admin_name = supply_chain_admin
  admin_password = $pwd_var
  first_name = $firstname_var
  last_name = $lastname_var 
  email = $email_var
  must_change_password = false
  edition = enterprise;

-- Create an organization account for admin purposes:
CREATE ORGANIZATION ACCOUNT hol_org_account
  admin_name = org_admin
  admin_password = $pwd_var
  first_name = $firstname_var
  last_name = $lastname_var 
  email = $email_var
  must_change_password = false
  edition = enterprise; 

-- Get an overview of all the accounts in the organization.
-- This SHOW command should return 3 rows:

SHOW ACCOUNTS;

In [ ]:
SHOW ACCOUNTS;

### Step 3 - 2つ目のアカウント `HOL_ACCOUNT2`の設定
1. 上記にリストアップされている、`HOL_ACCOUNT2`へアクセスし、`supply_chain_admin`ユーザーでログインします
2. 次のSQLを実行します

**account_url**が実際にアクセスするURLになります

```sql
-- Run this in hol_account2, logged in as supply_chain_admin user
-- Make sure you run this as ACCOUNTADMIN

USE ROLE accountadmin;

CREATE OR REPLACE WAREHOUSE compute_wh WAREHOUSE_SIZE=xsmall INITIALLY_SUSPENDED=TRUE;
GRANT ALL ON WAREHOUSE compute_wh TO ROLE public;

CREATE ROLE supply_chain_admin_role;
GRANT ROLE accountadmin TO ROLE supply_chain_admin_role; -- for simplicity in this lab
GRANT ROLE supply_chain_admin_role TO USER supply_chain_admin;

ALTER USER supply_chain_admin 
  SET DEFAULT_ROLE = supply_chain_admin_role;

USE ROLE supply_chain_admin_role;
CREATE DATABASE supply_chain_db;
```

### Step 4 - 組織アカウントの設定と初期アカウントの名前変更

1. 上記にリストアップされている、`HOL_ORG_ACCOUNT`へアクセスし、`supply_chain_admin`ユーザーでログインします
2. 次のSQLを実行します

**account_url**が実際にアクセスするURLになります

*名称変更後は、今アクセスしているこのノートブックのURLが変更になります*

```sql
USE ROLE accountadmin;

CREATE OR REPLACE WAREHOUSE compute_wh WAREHOUSE_SIZE=xsmall INITIALLY_SUSPENDED=TRUE;
GRANT ALL ON WAREHOUSE compute_wh TO ROLE public;

-- Rename the Primary Account:
USE ROLE globalorgadmin;


-- execute the following two commands together, 
-- no other commands in between:

  show accounts;
  SET my_curr_account = (SELECT "account_name" FROM TABLE(RESULT_SCAN(LAST_QUERY_ID())) order by "created_on" ASC LIMIT 1);

-- View and rename the account:
SELECT $my_curr_account;

ALTER ACCOUNT identifier($my_curr_account) 
  RENAME TO hol_account1 SAVE_OLD_URL = true;

-- Enable users with the ACCOUNTADMIN role to set up Cross-Cloud Auto-Fulfillment
SELECT SYSTEM$ENABLE_GLOBAL_DATA_SHARING_FOR_ACCOUNT('hol_account1');
SELECT SYSTEM$ENABLE_GLOBAL_DATA_SHARING_FOR_ACCOUNT('hol_account2');

SHOW ACCOUNTS;

-- You should see 3 rows similar to the image below.
```

### Step 5 - Sales, MarketingとSuppuly Chain Domainのプロファイル作成
組織アカウント`HOL_ORG_ACCOUNT`で`org_admin`ユーザーとして作業を継続し、データプロバイダープロファイルを作成します。営業、マーケティング、サプライチェーンの 3 つのビジネス ドメイン用のプロファイルを設定します。

```sql
USE ROLE globalorgadmin;
USE WAREHOUSE COMPUTE_WH;


CREATE OR REPLACE DATABASE OrgProfileDB;
CREATE OR REPLACE STAGE org_profile_stage;

-- Profile of the sales domain:
COPY INTO @org_profile_stage/profile_sales/manifest.yml
FROM (
  SELECT $$
title: "Sales"
description: "This is the profile of the global sales organization in the company. This includes account executives, sales engineers, sales analysts, and all the back office functions for the sales operations. Sales analysts and related back office teams are the primary owners of data products published by the sales domain.
\n
We provide various types of sales analytics data, sales forcast data and projections, as well as different types of historical sales analysis. We are also planning to share ML models as a product for teams to perform self-service sales forecast exercises.

\n - Business Domain Owner: James Kirk
\n - Domain Data Steward:  Leonard McCoy
\n - Lead Data Engineer:   Nyota Uhura
\n
 For data product Q&A please join the Slack channel:  #sales-data-products
\n For any other question please contact us at: captain.kirk@snowflake.com.
\n
"
contact: "youremail@whatever.com"
approver_contact: "youremail@whatever.com"
allowed_publishers:
  access:
    - all_internal_accounts: "true"
logo: "urn:icon:diamond:orange"
  $$
) SINGLE = TRUE
  OVERWRITE = TRUE
  FILE_FORMAT = (COMPRESSION = NONE ESCAPE_UNENCLOSED_FIELD = NONE);


-- Profile of the supply chain domain:  
COPY INTO @org_profile_stage/profile_supply/manifest.yml
FROM (
  SELECT $$
title: "Supply Chain"
description: "Welcome to the Supply Chain domain! This is the home of all supply chain related data products that we make available throughout the company. 
\n
The Supply Chain department is responsible for the end-to-end flow of our products, from the initial sourcing of raw materials through to the delivery of finished goods to our resellers. Our scope encompasses planning, procurement, manufacturing coordination, warehousing, transportation, and distribution. We are dedicated to ensuring efficiency, minimizing costs, mitigating risks, and ultimately ensuring production speed and quality through a reliable and optimized supply chain.
\n
Our Key Responsibilities Include:
\n	- Sourcing & Procurement: managing our network of suppliers to secure quality materials at competitive prices.
\n	- Logistics & Distribution: Managing warehousing, transportation, and delivery of materials and goods.
\n	- Inventory Management: Optimizing inventory levels to balance supply and demand while minimizing holding costs.
\n	- Supplier Relationship Management: Building and maintaining strong relationships with our key suppliers.
\n	- Supply Chain Optimization: Continuously identifying and implementing improvements to our processes and network.
\n	- Risk Management: Identifying and mitigating potential disruptions within the supply chain.
\n
To support other departments in making informed decisions, we maintain various types of data products, such as:
\n	- Inventory Reports & Dashboards: Real-time visibility into stock levels and potential shortages.
\n	- Order Status & Tracking: Information on the progress of purchase orders and shipment tracking details.
\n	- Supplier Performance Metrics: Data on supplier delivery performance, quality, and cost adherence.
\n	- Transportation & Logistics Reports: Data on shipping costs and on-time delivery rates.
\n  
Please reach out to the Supply Chain department if you have any other data needs related to our operations. We are here to support your success!
"
contact: "youremail@whatever.com"
approver_contact: "youremail@whatever.com"
allowed_publishers:
  access:
    - account: "HOL_ACCOUNT2"
logo: "urn:icon:blocks:aqua"
  $$
) SINGLE = TRUE
  OVERWRITE = TRUE
  FILE_FORMAT = (COMPRESSION = NONE ESCAPE_UNENCLOSED_FIELD = NONE);

-- Profile of the marketing domain:
COPY INTO @org_profile_stage/profile_marketing/manifest.yml
FROM (
  SELECT $$
title: "Marketing Team"
description: "Your Marketing Department: Driving Growth Through Customer Connection & Insights
\n\n
Who We Are: Your Marketing Department is dedicated to understanding our customers and the market to effectively promote our offerings, build our brand, and ultimately drive organizational growth. We are the team responsible for creating compelling messaging, engaging with our target audiences across various channels, and fostering strong customer relationships.
\n\nWhat Data We Provide: We are a key source of valuable data that informs strategic decisions across the organization. We provide insights on:

\n- Customer Understanding: Demographics, behaviors, preferences, feedback, and purchase history.
\n- Campaign Performance: Effectiveness of marketing initiatives, including reach, engagement, conversions, and ROI.
\n- Market & Competitive Landscape: Trends, competitor activities, and opportunities within our industry.
\n- Digital Performance: Website traffic, user behavior, and online engagement metrics.
\n- Lead Generation: Tracking and analyzing the flow of potential customers.
\n
How Our Data Helps You: Our data empowers informed decision-making in areas such as product development, sales strategies, customer service improvements, and overall business planning. By understanding our customers and the impact of our marketing efforts, we contribute directly to the company success!
\n"
contact: "youremail@whatever.com"
approver_contact: "youremail@whatever.com"
allowed_publishers:
  access:
    - account: "HOL_ACCOUNT1"
logo: "urn:icon:team:pink"
  $$
) SINGLE = TRUE
  OVERWRITE = TRUE
  FILE_FORMAT = (COMPRESSION = NONE ESCAPE_UNENCLOSED_FIELD = NONE);


-- Create the 3 profiles for the 3 domains:
CREATE ORGANIZATION PROFILE sales FROM @org_profile_stage/profile_sales/ PUBLISH = TRUE;

CREATE ORGANIZATION PROFILE supplychain FROM @org_profile_stage/profile_supply/ PUBLISH = TRUE;

CREATE ORGANIZATION PROFILE marketing FROM @org_profile_stage/profile_marketing/ PUBLISH = TRUE;

SHOW ORGANIZATION PROFILES;
```

## 組織リスティングの作成と発行

1. データ共有/Data Sharing > プロバイダーStudio/Provider Studioへアクセスし、`+リストを作成/+ Create listing`をクリック
2. `内部マーケットプレイス/Internal Marketplace`を選択
![listing](https://quickstarts.snowflake.com/guide/internal_marketplace_intra_org_sharing/img/4ed4ce2d2816957d.png)
3. `Untitled Listing`をクリックし、データ製品名を入力(今回はOrder Insight)し、**保存/Save**をクリック
![title](https://quickstarts.snowflake.com/guide/internal_marketplace_intra_org_sharing/img/293c2606948041ee.png)
4. **+ プロファイル/+ Profile**ボタンをクリックし、`Sales`を選択する
![profile](https://quickstarts.snowflake.com/guide/internal_marketplace_intra_org_sharing/img/7fdf4bddeb4b567a.png)

### 共有するデータオブジェクトの選択

5. **データ製品の追加/Add Data Product**をクリックして、オブジェクト Exploreを表示する
6. TB_101 > SYN_SCHEMA配下のテーブルを選択し、**完了/Done**を選択して**保存/Save**する


### アクセス権と認証プロセスの設定
7. **+アクセス管理/+Access Control**ボタンをクリックして、アクセスおよび探索の設定を行う
![access control](https://quickstarts.snowflake.com/guide/internal_marketplace_intra_org_sharing/img/833d6c223c3cd0b5.png)

デフォルトの設定は次の通り
- *Grant Access*: No accounts or role are pre-approved
- *Allow Discovery*: Entire Organization

上記の結果、すべてのデータ利用者はデータ製品を使用する承認を得るためにアクセスをリクエストする必要があります。**リクエスト承認フローの設定/Set up request Approval flow**をクリックして続行してください。

リクエスト承認プロセス用に外部ワークフローエンジンを設定することも可能です。本ハンズオンでは、外部ページを表示するようにします。(Manage Request outside of Snowflakeを選択)
![approval_flow1](https://quickstarts.snowflake.com/guide/internal_marketplace_intra_org_sharing/img/946ccee29912813.png)

承認フローの設定を確認した後、Snowflakeはさらに追加の設定が必要となります。
その理由は次のとおりです：
- このリストは組織全体から検出可能に設定されています。
- 組織に別のアカウントを追加したが、異なるクラウドリージョンにある場合はどうでしょうか？
- その場合、Snowflakeは転送コストを最小化するため、そのリージョンへの増分レプリケーションを透過的に実行します。
- データプロバイダーとして、このレプリケーションの頻度を選択できます。

では、(1) 設定を確認し、(2) レプリケーション間隔を毎日（1日）に変更し、(3) このリストの設定を保存しましょう：
![auto fulfilment](https://quickstarts.snowflake.com/guide/internal_marketplace_intra_org_sharing/img/855859896cf2112e.png)

### オプションのメタデータとSLOを追加する

データ製品はデータ利用者にとって理解しやすく信頼できるものであるべきです。製品を説明する追加のメタデータを追加しましょう（下図参照）。

- ビジネス**説明/Description**を追加してリスト内容を文書化します。
  - 例：「本データ製品は顧客注文の取引記録を含み、個々の注文詳細と購入された特定商品を関連付けます。注文ID、顧客識別子、注文日、商品名、数量、価格などの情報が含まれます。このデータを活用することで、顧客購買パターンの分析、人気商品の特定、注文頻度の把握、販売動向の洞察を得ることが可能です。」
- 追加情報のURLを提供してドキュメントを追加してください。（現時点では任意のURLを入力可、例：http://www.snowflake.com/data-mesh）
- 利用規約のURLを記載して利用規約を追加してください。
- データプロダクト所有者からデータ利用者への**サービスレベル目標(SLA)**を示す**属性を追加**してください。指定可能な項目：
  - **更新頻度**：データプロダクトの更新頻度（例：共有テーブルへの新規/更新レコード追加間隔）
  - **地理的カバレッジ**：複数地域でSnowflakeを利用している場合、このデータ製品を共有する地域。
  - **時間範囲**：含まれる履歴データの期間。
  - **タイムスタンプの粒度**：データポイント間の間隔。例：受注ごとに1レコードの場合は「イベントベース」、日付単位で集計する場合は「日次」など。

![Additional Info](https://quickstarts.snowflake.com/guide/internal_marketplace_intra_org_sharing/img/cd28ccb2f30c0f29.png)

- **Quick Start Example**を追加します。(今回は、TB_101.publicにある`Qucik Start Example`というノートブックを選択します)
- **Data Dictionary**を作成します。Snowflakeはデータプロダクト内の全オブジェクトについて、カラム情報とサンプルデータを自動的にコンパイルします。少なくとも1つ（最大5つまで）のデータオブジェクトを選択し、**+Add to Featured**をクリックしてください。これらはディクショナリで消費者が最初に目にするオブジェクトとなります。提案：Customer Loyalty、Order Header、Order Detailをフィーチャー対象として選択してください。

![Data Dictionary](https://quickstarts.snowflake.com/guide/internal_marketplace_intra_org_sharing/img/c3b72a1524599138.png)

### 内部マーケットプレイスへのリスト公開

右上の青い**公開/Publish**ボタンをクリックしてください。

これで、データ製品が公開されています。内部マーケットプレイスへ移動すると確認ができます。

![Internal Marketplace](https://quickstarts.snowflake.com/guide/internal_marketplace_intra_org_sharing/img/11bbfb5810f764e2.png)

### 消費アカウントからの確認

1. 上記にリストアップされている、`HOL_ACCOUNT2`へアクセスし、`supply_chain_admin`ユーザーでログインします
2. カタログ/Catalog > 内部マーケットプレイス/Internal Markeplace へアクセスし表示されていることを確認
3. クリックして、どのように表示されるかを確認してください。